# ETL Project

## Dependencies

In [1]:
import pandas as pd
import datetime
import requests
from sqlalchemy import create_engine

## Extraction

### Climate Data

In [2]:
# Api Key
key = '22bbc46976cc45e8b1104224200104'

# List of dates corresponding to the month and the year (The API only manages to get a month at a time)
dates = ['2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01']
end_date = ['2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30']
# Create an empty list to store all the data
date_list = []
avgtempC_list = []
humidity_list = []
FeelsLikeC_list = []
city_name = []

# Latitude and Longitude of the sites
latitud = ['19.422614', '25.678396', '21.175957', '20.678767']
longitud = ['-99.136465', '-100.320128', '-86.835151', '-103.346187']
city = [1, 2, 3, 4]

site_flag = 0

for lat in latitud:

    end_date_flag = 0
    # For each start and end date, get the URL. Use a flag to iterate through the end_date list.
    for date in dates:
        # Built the URL.
        url='http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={key}&q={latitud},%{longitud}&format=json&date={date}&enddate={end_date}&includelocation=yes&tp=24'.format(
        key=key,latitud=lat,longitud=longitud[site_flag],date=date,end_date=end_date[end_date_flag])
        end_date_flag+=1
        # Get response from the URL.
        response = requests.get(url).json()
        # Create a flag to iterate thourgh the hole month.
        month_day = 0
        # For each element in the JSON (day of the mont), extract the Date, Avg Temperature C,
        # Humidity %, Feels Like C.
        for i in response['data']['weather']:
            # Append the element of the dictionary that corresponds to each list. 
            date_list.append(response['data']['weather'][month_day]['date'])
            avgtempC_list.append(float(response['data']['weather'][month_day]['avgtempC']))
            humidity_list.append(float(response['data']['weather'][month_day]['hourly'][0]['humidity']))
            FeelsLikeC_list.append(float(response['data']['weather'][month_day]['hourly'][0]['FeelsLikeC']))
            city_name.append(city[site_flag])
            month_day+=1

    # Create a dictionary to store the lists
    dict_climate = {'Date':date_list, 
                    'Temp':avgtempC_list,
                    'Hum':humidity_list,
                    'FLike':FeelsLikeC_list,
                    'City': city_name
                   }
    print(lat + ', ' + longitud[site_flag] + ', ' + str(city[site_flag]))
    site_flag+=1

# Create the DataFrame based on the previous dictionary
df_climate = pd.DataFrame(dict_climate)
df_climate["Date"] = pd.to_datetime(df_climate["Date"])
df_climate['Date'] = df_climate['Date'].dt.strftime('%d/%m/%Y')
df_climate.info()

df_climate.head()

19.422614, -99.136465, 1
25.678396, -100.320128, 2
21.175957, -86.835151, 3
20.678767, -103.346187, 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 5 columns):
Date     728 non-null object
Temp     728 non-null float64
Hum      728 non-null float64
FLike    728 non-null float64
City     728 non-null int64
dtypes: float64(3), int64(1), object(1)
memory usage: 28.6+ KB


,Date,Temp,Hum,FLike,City
0,01/11/2019,18.0,65.0,16.0,1
1,02/11/2019,17.0,70.0,16.0,1
2,03/11/2019,17.0,64.0,16.0,1
3,04/11/2019,19.0,59.0,18.0,1
4,05/11/2019,18.0,66.0,17.0,1


### Site Information

In [3]:
# Load the data from a CSV file
sites_df = pd.read_csv('sites.csv', encoding='utf-8')
sites_df.info()
sites_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
site_id         10 non-null int64
site_name       10 non-null object
client_id       10 non-null int64
tarifa_cfe      10 non-null object
tarifa2         10 non-null object
division_cfe    10 non-null object
tarifa3         0 non-null float64
zona            10 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 768.0+ bytes


,site_id,site_name,client_id,tarifa_cfe,tarifa2,division_cfe,tarifa3,zona
0,15099,Restaurante 01,1,GDMTH,HM,Jalisco,NaN,GDL
1,36983,Restaurante 02,1,GDMTH,HM,Golfo Norte,NaN,MTY
2,38716,Restaurante 03,1,GDMTH,HM,Peninsular,NaN,CUN
3,26804,Restaurante 04,1,GDMTH,HM,Valle de México Centro,NaN,CDMX
4,32703,Restaurante 05,1,GDMTH,HM,Golfo Norte,NaN,MTY


### Region Information

In [4]:
# Create a dictionary, based on the data we have
region = {'region_id': [1,2,3,4], 'region_name': ['CDMX', 'MTY', 'CUN', 'GDL']}

In [5]:
# Create the data frame from the previous dictionary
region_df = pd.DataFrame(region)
region_df

,region_id,region_name
0,1,CDMX
1,2,MTY
2,3,CUN
3,4,GDL


### Energy Consumption

In [6]:
# Load the data from a CSV file
measurements_df = pd.read_csv('measurements.csv')
measurements_df.info()
measurements_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 844013 entries, 0 to 844012
Data columns (total 8 columns):
device_id      844013 non-null int64
power(kW)      844013 non-null float64
energy(kWh)    844013 non-null float64
site_id        844013 non-null int64
category       844013 non-null object
device_name    844013 non-null object
Fecha          844013 non-null object
Hora           844013 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 51.5+ MB


,device_id,power(kW),energy(kWh),site_id,category,device_name,Fecha,Hora
0,44989,3.2694,0.817350,15099,Ilum,Iluminacion A,01/11/2019,06:00:00 p. m.
1,44989,3.2839,0.820975,15099,Ilum,Iluminacion A,01/11/2019,06:15:00 p. m.
2,44989,3.6099,0.902475,15099,Ilum,Iluminacion A,01/11/2019,06:30:00 p. m.
3,44989,3.3515,0.837875,15099,Ilum,Iluminacion A,01/11/2019,06:45:00 p. m.
4,44989,3.2893,0.822325,15099,Ilum,Iluminacion A,01/11/2019,07:00:00 p. m.


## Sales Information 

In [7]:
# Load the data from a CSV file
sales_df = pd.read_csv('sales.csv')
sales_df.head()

,site_id,fecha,ventas,invitados
0,32703,01/01/2020,25627.53,118
1,15099,01/01/2020,52417.82,227
2,19865,01/01/2020,39135.47,170
3,23623,01/01/2020,30524.01,124
4,26800,01/01/2020,27009.54,113


## Transformation

### Climate Data

In [8]:
# Rename the columns so they coincide with the Postgre DB
df_climate.columns = ['date', 'avg_temp', 'humidity', 'f_like', 'region_id']
df_climate.head()

,date,avg_temp,humidity,f_like,region_id
0,01/11/2019,18.0,65.0,16.0,1
1,02/11/2019,17.0,70.0,16.0,1
2,03/11/2019,17.0,64.0,16.0,1
3,04/11/2019,19.0,59.0,18.0,1
4,05/11/2019,18.0,66.0,17.0,1


### Site Information

In [9]:
# List the desired sites by their site_id
sites_id = [15099, 36983, 38716, 26804]

# Filter the sites_df Data Frame by using 'isin' function
sites_df = sites_df[sites_df['site_id'].isin(sites_id)]

In [10]:
# Extract the desired columns
sites_df = sites_df[['site_id', 'site_name', 'zona']]
sites_df.head()

,site_id,site_name,zona
0,15099,Restaurante 01,GDL
1,36983,Restaurante 02,MTY
2,38716,Restaurante 03,CUN
3,26804,Restaurante 04,CDMX


In [11]:
# Rename the columns
sites_df.columns = ['site_id', 'site_name', 'region_name']
sites_df.head()

,site_id,site_name,region_name
0,15099,Restaurante 01,GDL
1,36983,Restaurante 02,MTY
2,38716,Restaurante 03,CUN
3,26804,Restaurante 04,CDMX


In [12]:
# Join sites_df with region_df on region_name column
sites_df = sites_df.merge(region_df, how='inner', on='region_name')
# From the previous "Join", extract the desired columns
sites_df = sites_df[['site_id', 'site_name', 'region_id']]
sites_df

,site_id,site_name,region_id
0,15099,Restaurante 01,4
1,36983,Restaurante 02,2
2,38716,Restaurante 03,3
3,26804,Restaurante 04,1


### Energy Consumption Part 1

In [13]:
# Filter measurements_df by the desired sites
measurements_df = measurements_df[measurements_df['site_id'].isin(sites_id)]

In [14]:
# Select the desired columns
measurements_df = measurements_df[['site_id', 'Fecha', 'category', 'energy(kWh)']]
measurements_df.head()

,site_id,Fecha,category,energy(kWh)
0,15099,01/11/2019,Ilum,0.817350
1,15099,01/11/2019,Ilum,0.820975
2,15099,01/11/2019,Ilum,0.902475
3,15099,01/11/2019,Ilum,0.837875
4,15099,01/11/2019,Ilum,0.822325


In [15]:
# Rename the columns
measurements_df.columns = ['site_id', 'date', 'category', 'energy_kWh']
measurements_df.head()

,site_id,date,category,energy_kWh
0,15099,01/11/2019,Ilum,0.817350
1,15099,01/11/2019,Ilum,0.820975
2,15099,01/11/2019,Ilum,0.902475
3,15099,01/11/2019,Ilum,0.837875
4,15099,01/11/2019,Ilum,0.822325


In [16]:
# Group the energy column by date
measurements_grouped = measurements_df.groupby(['site_id', 'date', 'category'], as_index=False).sum()
measurements_grouped.head()

,site_id,date,category,energy_kWh
0,15099,01/01/2020,Cocina,178.777400
1,15099,01/01/2020,Ilum,72.054150
2,15099,01/01/2020,Iny_Ext,93.290125
3,15099,01/01/2020,Refri,92.532600
4,15099,01/02/2020,Cocina,217.115400


### Category Information Part 1

In [17]:
# From the measurements table, get the unique categories from categoy column
category_list = measurements_grouped['category'].unique().tolist()
category_list

['Cocina', 'Ilum', 'Iny_Ext', 'Refri', 'Hvac', 'Contactos', 'Mach']

In [18]:
# Create a list of id's for each category
id_list = []
id = 1
for i in category_list:
    id_list.append(id)
    id+=1
id_list

[1, 2, 3, 4, 5, 6, 7]

In [19]:
# Create a dictionary from the id_list and category_list
category_dict = {'category_id':id_list, 'category':category_list}
# Create a category from the decitionaries
category_df = pd.DataFrame(category_dict)
category_df

,category_id,category
0,1,Cocina
1,2,Ilum
2,3,Iny_Ext
3,4,Refri
4,5,Hvac
5,6,Contactos
6,7,Mach


### Energy Consumption & Category Information Part 2

In [20]:
# # Join measurements_df with category_df on region_name column
measurements_df = measurements_grouped.merge(category_df, how='inner', on='category')
measurements_df.head()

,site_id,date,category,energy_kWh,category_id
0,15099,01/01/2020,Cocina,178.77740,1
1,15099,01/02/2020,Cocina,217.11540,1
2,15099,01/03/2020,Cocina,195.89090,1
3,15099,01/04/2020,Cocina,125.66560,1
4,15099,01/11/2019,Cocina,78.34855,1


In [21]:
# From the previous df, extract the desired columns
measurements_df = measurements_df[['site_id', 'date', 'category_id', 'energy_kWh']]
measurements_df.head()

,site_id,date,category_id,energy_kWh
0,15099,01/01/2020,1,178.77740
1,15099,01/02/2020,1,217.11540
2,15099,01/03/2020,1,195.89090
3,15099,01/04/2020,1,125.66560
4,15099,01/11/2019,1,78.34855


In [22]:
# Change column 'date' type to datetime
measurements_df['date'] = pd.to_datetime(measurements_df['date'])
measurements_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2904 entries, 0 to 2903
Data columns (total 4 columns):
site_id        2904 non-null int64
date           2904 non-null datetime64[ns]
category_id    2904 non-null int64
energy_kWh     2904 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 113.4 KB


In [23]:
# Rename category_df columns
category_df.columns = ['category_id', 'category_name']
category_df

,category_id,category_name
0,1,Cocina
1,2,Ilum
2,3,Iny_Ext
3,4,Refri
4,5,Hvac
5,6,Contactos
6,7,Mach


### Sales Data

In [24]:
# Filter the df by using 'isin' function
sales_df = sales_df[sales_df['site_id'].isin(sites_id)]

In [25]:
# Rename the columns
sales_df.columns = ['site_id', 'date', 'sales', 'customers']
sales_df.head()

,site_id,date,sales,customers
1,15099,01/01/2020,52417.82,227
6,15099,02/01/2020,37879.17,181
11,15099,03/01/2020,45017.42,201
16,15099,04/01/2020,58530.52,266
21,15099,05/01/2020,51642.40,260


## Load

In [33]:
# Ask for the  postgres password
password = input("Postgres password: ")

Postgres password: postgres


In [27]:
# Create the engine
connection_string = "postgres:"+ password +"@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{connection_string}')

In [28]:
# Display the found tables
engine.table_names()

['sites', 'measurements', 'categories', 'region', 'meteorological', 'sales']

### Load the df to the corresponding tables

In [29]:
region_df.to_sql(name='region', con=engine, if_exists='append', index=False)

In [30]:
category_df.to_sql(name='categories', con=engine, if_exists='append', index=False)

In [31]:
df_climate.to_sql(name='meteorological', con=engine, if_exists='append', index=False)

In [36]:
sites_df.to_sql(name='sites', con=engine, if_exists='append', index=False)

In [37]:
measurements_df.to_sql(name='measurements', con=engine, if_exists='append', index=False)

In [38]:
sales_df.to_sql(name='sales', con=engine, if_exists='append', index=False)